In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from ortools.sat.python import cp_model
import graphviz
import numpy as np
from time import time
from itertools import combinations
from itertools import product
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import networkx as nx
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
import col1
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.neighbors import KNeighborsClassifier
from gurobipy import *

In [6]:
def buildTree(y,Cl,K,L,V,M,X,Y,colnames,
              classnames=["Mammal","Bird","Reptile","Fish","Amphibian","Bug","Invertebrate"],must_link=None):
    indent=0
    Tnx=nx.DiGraph()
    root_attribute=None
    flag=False
    l=0
    while not(flag) and l<M:
        y_array=[y[(0,0,l,p)].x for p in range(V)]
        if 1 in y_array:
            root_attribute=l
            flag=True
        l+=1
    (values,counts) = np.unique(Y,return_counts=True)
    values=list(values)
    ind=np.argmax(counts) 
    freq=[counts[values.index(i)] if i in values else 0 for i in range(len(classnames))]
    Tnx.add_node(indent,
                 label='best attribute='+colnames[root_attribute],attribute=root_attribute,samples=len(Y),
                 values=freq,value=None,instances=list(range(len(Y))), labels=Y, classe=classnames[ind],color=col1.get_color(freq,len(classnames)))
    for i in range(L):
        node=0
        j=0
        flag=False
        while not(flag) and j<K:
            attribute=-1
            value=-1
            flag1=False
            l=0
            parent=node
            while not(flag1) and l<M:
                y_array=[y[(i,j,l,p)].x for p in range(V)]
                if 1 in y_array:
                    attribute=l
                    value=y_array.index(0)
                    flag1=True
                l+=1
            if not(flag1):
                flag=True
                if must_link and i in must_link.keys():
                    Tnx.nodes[node]['best_attribute']="same_leaf_constraint="+str(must_link[i])+"\n"
                else:
                    Tnx.nodes[node]['best_attribute']=""
                classe=[Cl[(i,c)].x for c in range(len(C))]
                Tnx.nodes[node]['class']=classe.index(1)
            else:
                neighbors=[i for i in Tnx.neighbors(node)]
                q=0
                drap=False
                while not(drap) and q<len(neighbors):
                    if Tnx.nodes[neighbors[q]]['attribute']==attribute and Tnx.nodes[neighbors[q]]['value']==value:
                        node=neighbors[q]
                        drap=True
                    q+=1
                if not(drap):
                    indent+=1
#                     print(Tnx.nodes[parent]['instances'])
                    indices_t=np.where(np.take(X_train[:,attribute], Tnx.nodes[node]['instances'])<=value)[0] 
                    instances_t=np.take(Tnx.nodes[node]['instances'],indices_t)
                    labels_t=np.take(Tnx.nodes[node]['labels'],indices_t)
                    #print(labels_t,indices_t,instances_t,node, parent,'attribute',attribute,value)
                    (values_t,counts_t) = np.unique(labels_t,return_counts=True)
                    values_t=list(values_t)
                    freq_t=[counts_t[values_t.index(i)] if i in values_t else 0 for i in range(len(classnames))]
                    color_t=None
                    if len(counts_t)==0:
                        color_t='black'
                        class_t='None'
                    else:
                        ind_t=np.argmax(freq_t)
#                         print(freq_t,counts_t,values_t,len(values_t),len(counts_t))
                        color_t=col1.get_color(freq_t,len(classnames))
                        class_t=classnames[ind_t]
                    Tnx.add_node(indent,instances=instances_t, labels=labels_t,samples=len(instances_t),
                                 attribute=attribute,value=value,values=freq_t,classe=class_t,color=color_t)
                    Tnx.add_edge(node,indent)
                    Tnx.nodes[node]['best_attribute']="attribute="+colnames[attribute]+"\n"
                    node=indent
                    if j==K-1:
                        if must_link and i in must_link.keys():
                            Tnx.nodes[node]['best_attribute']="same_leaf_constraint="+str(must_link[i])+"\n"
                        else:
                            Tnx.nodes[node]['best_attribute']=""
                        classe=[Cl[(i,c)].x for c in range(len(classnames))]
                        Tnx.nodes[node]['class']=classe.index(1)
            j+=1
    return Tnx

In [3]:
def export_graphviz_cp(tree):
    Tree=graphviz.Digraph(format='png',graph_attr={"randir":"LR"},
                                    node_attr={'shape':"box"})
    indent=0
    list_nodes=[0]
    Tree.node(str(indent),
                      label=tree.nodes[0]['best_attribute']+'samples='+str(tree.nodes[0]['samples'])+'\nvalue='+str(tree.nodes[0]['values'])+'\nclass='+tree.nodes[0]['classe'],
                     fillcolor=tree.nodes[0]['color'],style="rounded,filled")
    here={}
    here[indent]=indent
    while len(list_nodes)>=1:
        node=list_nodes.pop(0)
        neighbors=[i for i in tree.neighbors(node)]
        if len(neighbors)!=0:
            node1=neighbors[0] if tree.nodes[neighbors[0]]['value']!=0 else neighbors[1]
            node2=neighbors[0] if tree.nodes[neighbors[0]]['value']==0 else neighbors[1]
            indent+=1
            here[node1]=indent
            Tree.node(str(indent),
                      label=tree.nodes[node1]['best_attribute']+'samples='+str(tree.nodes[node1]['samples'])+'\nvalue='+str(tree.nodes[node1]['values'])+'\nclass='+tree.nodes[node1]['classe'],
                     fillcolor=tree.nodes[node1]['color'],style="rounded,filled")
            Tree.edge(str(here[node]),str(indent),label='\nvalue=1')
            indent+=1
            here[node2]=indent
            Tree.node(str(indent),
                      label=tree.nodes[node2]['best_attribute']+'samples='+str(tree.nodes[node2]['samples'])+'\nvalue='+str(tree.nodes[node2]['values'])+'\nclass='+tree.nodes[node2]['classe'],
                     fillcolor=tree.nodes[node2]['color'],style="rounded,filled")
            Tree.edge(str(here[node]),str(indent),label='\nvalue=0')
            if tree.out_degree(node1)!=0:
                list_nodes.append(node1)
            if tree.out_degree(node2)!=0:
                list_nodes.append(node2)

                
    return Tree

In [17]:
def decisionTreeConstraint(K,L,V,max_time,X,Y,classnames,colnames,imbalanced=True,min_number=1,precedence=None
                           ,must_link=None,max_cost=False,list_cost=[],attribute_exclusion=None):
    model = Model("mip1")
    N=X.shape[0]
    M=X.shape[1]
    x={}
    y={}
    cl={}
    #z={}
    #s={}
    u={}
    r={}
    z={}
    M=X.shape[1]
    for i in range(L):
        for j in range(K):
            for l in range(M):
                for p in range(V):
                    y[(i,j,l,p)]=model.addVar(lb=0,ub=1,vtype=GRB.BINARY, name='('+str(i)+','+str(j)+','+str(l)+','+str(p)+')')
                    for a in range(L):
                        u[(i,j,l,p,a)]=model.addVar(lb=0,ub=1,vtype=GRB.BINARY,name='u'+'('+str(i)+','+str(j)+','+str(l)+','+str(p)+str(a)+')')
            for a in range(L):
                for q in range(3):
                    z[(i,j,a,q)]=model.addVar(lb=0,ub=1,vtype=GRB.BINARY,name='z'+'('+str(i)+','+str(j)+','+str(a)+','+str(q)+')')
    att_value={}
    att_value_entr={}
    for e in range(N):
        for i in range(L):
            x[(e,i)]=model.addVar(lb=0,ub=1,vtype=GRB.BINARY,name='x'+'('+str(e)+','+str(i)+')')
        for c in range(len(C)):
            r[(e,c)]=model.addVar(lb=0,ub=1,vtype=GRB.BINARY,name='r'+'('+str(e)+','+str(c)+')')


    for i in range(L):
        for c in range(len(C)):
            cl[(i,c)]=model.addVar(lb=0,ub=1,vtype=GRB.BINARY,name='cl'+'('+str(i)+','+str(c)+')')

    #Constraints
    for i in range(L):
        model.addConstr(sum(y[(i,0,l,p)] for l in range(M) for p in range(V))>=1)
        for j in range(K):
            #Ensure that exactly no more than one variable is chosen
            model.addConstr(sum(y[(i,j,l,p)] for l in range(M) for p in range(V))<=1)
            if j!=0:
                model.addConstr(sum(y[(i,j,l,p)] for l in range(M) for p in range(V)) <= sum(y[(i,j-1,l,p)] for l in range(M) for p in range(V)))
    #         #Ensure that the value chosen corresponds to the variable chosen
    #         for l in range(m):
    #             model.Add(sum(y[(i,j,l,p)] for p in range(V))==x[(i,j,l)])

        for l in range(M):
            #Ensure that exactly each variable is chosen nomore than once in a branch
            model.addConstr(sum(y[(i,j,l,p)] for j in range(K) for p in range(V))<=1)

    #Ensure the existence of the complentary branch
    for i in range(L):
        L_i=[b for b in range(L) if b!=i]
        for a in L_i:
            model.addConstr(sum(u[(i,j,l,p,a)] for j in range(K) for l in range(M) for p in range(V))>=1)
            for l in range(M):
                model.addConstr(sum(y[(i,0,l,p)] for p in range(V))==sum(y[(a,0,l,p)] for p in range(V)))
        for j in range(K):
            for l in range(M):
                for p in range(V):
                    #model.Add(u[(i,j,l,p,i)]==0)
                    for a in range(L):
                        model.addGenConstrIndicator(u[(i,j,l,p,a)],0,y[(i,j,l,p)]==y[(a,j,l,p)])
#                         model.addGenConstrIndicator(u[(i,j,l,p,a)],1,y[(i,j,l,p)]==1-y[(a,j,l,p)])

                        model.addConstr(u[(i,j,l,p,a)] + max_([y[(i,j,l,p)], y[(a,j,l,p)]]) - min_([y[(i,j,l,p)], y[(a,j,l,p)]])<= 1 )

        for j in range(K):
            for a in range(L):
                model.addConstr(z[(i,j,a,1)]==z[(a,j,i,1)])
                if j!=0:
                    model.addGenConstrIndicator(z[(i,j,a,1)],1,sum(u[(i,j1,l,p,a)] for j1 in range(j) for l in range(M) for p in range(V))==0)
                    model.addGenConstrIndicator(z[(i,j,a,2)],1,sum(u[(i,j1,l,p,a)] for j1 in range(j) for l in range(M) for p in range(V))>=1)
                model.addGenConstrIndicator(z[(i,j,a,0)],1,sum(u[(i,j1,l,p,a)] for j1 in range(j+1) for l in range(M) for p in range(V))==0)
                model.addGenConstrIndicator(z[(i,j,a,1)],1,sum(u[(i,j,l,p,a)] for l in range(M) for p in range(V))>=1)
                model.addConstr(sum(z[(i,j,a,q)] for q in range(3))==1)
                for l in range(M):
                    model.addGenConstrIndicator(z[(i,j,a,1)],1,sum(y[(i,j,l,p)] for p in range(V))==sum(y[(a,j,l,p)] for p in range(V)))
            model.addConstr(sum(z[(i,j,a,1)] for a in L_i)>=sum(y[(i,j,l,p)] for l in range(M) for p in range(V)))
        for j in range(K-1):
            for a in L_i:
                model.addConstr(z[(i,j,a,0)]<=sum(y[(i,j+1,l,p)] for l in range(M) for p in range(V)))
    for e in range(N):
        model.addConstr(sum(x[(e,i)] for i in range(L))==1)
        for i in range(L):
            for j in range(K):
                for l in range(M):
                    val= [1,0] if X[e,l]==0 else [0,1]
                    for p in range(V):
                        model.addGenConstrIndicator(y[(i,j,l,p)],1,x[(e,i)]<=1-val[p]+y[(i,j,l,p)])
                        model.addGenConstrIndicator(y[(i,j,l,p)],1,x[(e,i)]<=1+val[p]-y[(i,j,l,p)])

    for i in range(L):
        model.addConstr(sum(cl[(i,c)] for c in range(len(C)))==1)
        model.addConstr(sum(x[(e,i)] for e in range(N))>=min_number)
        for c in range(len(C)):
            for cp in range(len(C)):
                model.addGenConstrIndicator(cl[(i,c)],1,sum(x[(e,i)]*y_train_ohot[e,c] for e in range(N))>=sum(x[(e,i)]*Y[e,cp] for e in range(N)))
    for e in range(N):
        for c in range(len(C)):
            #model.Add(sum(r[(e,c)] for c in range(len(C)))==1)
            for i in range(L):
                model.addGenConstrIndicator(x[(e,i)],1,r[(e,c)]>=-cl[(i,c)]+Y[e,c])
                #model.addGenConstrIndicator(x[(e,i)],1,r[(e,c)]>=cl[(i,c)]-Y[e,c])
                model.addGenConstrIndicator(x[(e,i)],1,r[(e,c)]<=2-cl[(i,c)]-Y[e,c])
                #model.addGenConstrIndicator(x[(e,i)],1,r[(e,c)]<=cl[(i,c)]+Y[e,c])
    if imbalanced:
        for c in range(len(C)):
            model.addConstr(sum(cl[(i,c)] for i in range(L))>=1)
    
    #ordering constraint
    if precedence:
        for pair in precedence:
            for i in range(L):
                for j in range(1,K):
                    model.addConstr(sum(y[(i,j1,pair[1],p)] for j1 in range(j) for p in range(V))<=1-sum(y[(i,j,pair[0],p)] for p in range(V)))
    if must_link: 
        for se in must_link:
            for e in se:
                for i in range(L):
                    model.addGenConstrIndicator(x[(e,i)],1,sum(x[(ep,i)] for ep in se)==len(se))
    if max_cost:
        for i in range(L):
            model.addConstr(sum(y[(i,j,l,p)]*list_cost[l] for j in range(K) for l in range(M) for p in range(V))<=max_cost)
            
    if attribute_exclusion:
        for att_list in attribute_exclusion:
            model.addConstr(sum(y[(i,j,at,p)] for i in range(L) for j in range(K) for at in att_list for p in range(V))<=len(att_list)-1)
            
    #Create a solver and solve.
#     solver = cp_model.CpSolver()
    model.setObjective(sum(r[(e,c)] for e in range(N) for c in range(len(C))),GRB.MINIMIZE)
    model.Params.timeLimit = max_time
#    model.Params.solutionLimit=1
    model.optimize()
    model.write("test.lp")
#     solver.parameters.linearization_level=0
#     solver.parameters.cp_model_probing_level=0
#     solver.parameters.num_search_workers=10
#     status = solver.Solve(model)
    filename='visual_cp_bis/dt_ortools_leaves='+str(L)+'_depth='+str(K)+'_time='+str(max_time)+'s'
    if model.status != GRB.Status.INFEASIBLE:
        dico={}
        if must_link:
            for se in must_link:
                for i in range(L):
                    if x[(se[0],i)].x==1:
                        dico[i]=se
        tree=buildTree(y=y,Cl=cl,K=K,L=L,V=V,M=M,colnames=colnames,classnames=classnames,X=X,Y=np.argmax(Y,axis=1),must_link=dico)
        if model.status == GRB.Status.OPTIMAL:
#                             print("Example: ",e,", branch: ",i)
#                             for j in range(K):
#                                 for l in range(M):
#                                     if sum(solver.Value(y[(i,j,l,p)]) for p in range(V))==1:
#                                         print("\t attribute: ",colnames[l],", Value: ",X[e,l])
            return tree,'Optimal'
        else:
            return tree, 'Not_Optimal'
    else:
        return None, 'No_Solution'

In [8]:
def getClass(tree, node,features):
        if tree.out_degree(node)==0:
            #print(node)
            return tree.nodes[node]['class']
        else :
            p=0
            flag=True
            nextnode=-1
            neighbors=[t for t in tree.neighbors(node)]
            while flag and p<= len(neighbors):
                  if features[tree.nodes[neighbors[p]]['attribute']]==tree.nodes[neighbors[p]]['value']:
                      nextnode=neighbors[p]
                      #print(node,nextnode,tree.nodes[neighbors[p]]['attribute'],features[tree.nodes[neighbors[p]]['attribute']])
                      #print(tree.nodes[neighbors[p]]['value'])
                      flag=False
                  p+=1
            return getClass(tree,nextnode,features) 
    
    #Just returning the class
def predict(tree,x):
    return getClass(tree,0,x)

In [9]:
dataset=pd.read_table("repo/constraint_dt/datasets/zoo/zoo.data",delimiter=',',names=['hair','feathers','eggs','milk','airborne','aquatic' ,'predator' ,'toothed','backbone','breathes','venomous','fins','legs','tail','domestic',
                       'catsize','type'])
Y =dataset['type'].as_matrix()
dm=pd.get_dummies(dataset['legs'],drop_first=True,prefix='legs')

dataset=dataset.drop('legs',axis=1)
dataset=dataset.drop('type',axis=1)
dataset=pd.concat([dataset,dm],axis=1)

X =dataset.as_matrix()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=0, stratify=Y)

C=np.unique(y_train)
lb=LabelBinarizer()
lb.fit(C)
y_train_ohot=lb.transform(y_train)

M=X_train.shape[1]
N=X_train.shape[0]
variables=[]
values=[]
cond_entropy=[]
unique=[np.unique(X_train[:,i]) for i in range(M)]
dataset.head()

/home/gnanfack/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  
/home/gnanfack/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/gnanfack/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.


,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,tail,domestic,catsize,legs_2,legs_4,legs_5,legs_6,legs_8
aardvark,1,0,0,1,0,0,1,1,1,1,0,0,0,0,1,0,1,0,0,0
antelope,1,0,0,1,0,0,0,1,1,1,0,0,1,0,1,0,1,0,0,0
bass,0,0,1,0,0,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0
bear,1,0,0,1,0,0,1,1,1,1,0,0,0,0,1,0,1,0,0,0
boar,1,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,1,0,0,0


In [18]:
l=7
k=5
max_time=120
v=2
test_accuracy={}
train_accuracy={}
tree,optimal=decisionTreeConstraint(L=l,K=k,V=v,max_time=max_time,X=X_train,Y=y_train_ohot,
                             colnames=list(dataset),
                            classnames=["Mammal","Bird","Reptile","Fish","Amphibian","Bug","Invertebrate"])
print(optimal)
tree1=export_graphviz_cp(tree)
#name=filename+'leaves='+str(l)+'_depth='+str(k)+'_maxtime='+str(max_time)+optimal
y_pre_test=[1+predict(tree,X_test[i,:]) for i in range(X_test.shape[0])]
y_pre_train=[1+predict(tree,X_train[i,:]) for i in range(X_train.shape[0])]
test_accuracy[(l,k)]=accuracy_score(y_pre_test,y_test)
train_accuracy[(l,k)]=accuracy_score(y_pre_train,y_train)
print("Leaves: ",l, ", Depth: ",k, ", test accuray: ", test_accuracy[(l,k)],", train accuracy: ", train_accuracy[(l,k)])
tree1

GurobiError: Unsupported type (<class 'gurobipy.gurobipy.GenExpr'>) for LinExpr addition argument

In [ ]:
tree1=export_graphviz_cp(tree)
#name=filename+'leaves='+str(l)+'_depth='+str(k)+'_maxtime='+str(max_time)+optimal
y_pre_test=[1+predict(tree,X_test[i,:]) for i in range(X_test.shape[0])]
y_pre_train=[1+predict(tree,X_train[i,:]) for i in range(X_train.shape[0])]
test_accuracy[(l,k)]=accuracy_score(y_pre_test,y_test)
train_accuracy[(l,k)]=accuracy_score(y_pre_train,y_train)
print("Leaves: ",l, ", Depth: ",k, ", test accuray: ", test_accuracy[(l,k)],", train accuracy: ", train_accuracy[(l,k)])
tree1

In [ ]:
optimal

In [ ]:
dataset=pd.read_table("car.data",delimiter=',',names=["buying","maint","doors","persons","lug_boot","safety","class"])
#X =np.asarray(dataset.values[:,0:dataset.shape[1]]-1,dtype=np.str)

Y =dataset['class'].as_matrix()
dataset=dataset.drop('class',axis=1)

mutiva_att=['buying','maint','doors','persons','lug_boot','safety']

for att in mutiva_att:
    dm1=pd.get_dummies(dataset[att],drop_first=True,prefix=att)
    dataset=dataset.drop(att,axis=1)
    dataset=pd.concat([dataset,dm1],axis=1)


X=dataset.as_matrix(columns=None)


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=0, stratify=Y)
C=np.unique(y_train)
lb=LabelBinarizer()
#lb.fit(C)
lb.fit(C)
y_train_ohot=lb.transform(y_train)
#y_train_ohot= np.hstack((1-y_train_ohot, y_train_ohot))
y_test_ohot=lb.transform(y_test)
#y_test_ohot= np.hstack((1-y_test_ohot, y_test_ohot))

N=X_train.shape[0]
M=X_train.shape[1]

dataset.head()

In [ ]:
## v=2
leaves=[6]
depths={6:[4]}
test_accuracy={}
train_accuracy={}
max_time={4:3600}
filename='datasets/car/constant_time/car_'
for l in leaves:
    for k in depths[l]:
        tree,optimal=decisionTreeConstraint(L=l,K=k,V=v,max_time=max_time[k],X=X_train,Y=y_train_ohot,
                             colnames=list(dataset),
                        classnames=["unacc", "acc", "good", "vgood"],imbalanced=False)
        print(optimal)
        tree1=export_graphviz_cp(tree)
        y_pre_test=[predict(tree,X_test[i,:]) for i in range(X_test.shape[0])]
        y_pre_train=[predict(tree,X_train[i,:]) for i in range(X_train.shape[0])]
        test_accuracy[(l,k)]=accuracy_score(y_pre_test,np.argmax(y_test_ohot,axis=1))
        train_accuracy[(l,k)]=accuracy_score(y_pre_train,np.argmax(y_train_ohot,axis=1))
        #name=filename+'leaves='+str(l)+'_depth='+str(k)+'_maxtime='+str(max_time[k])+'_'+optimal+'_train_acc='+str(round(train_accuracy[(l,k)],5))+'_test_acc='+str(round(test_accuracy[(l,k)],5))
        print("Leaves: ",l, ", Depth: ",k, ", test accuracy: ", test_accuracy[(l,k)],", train accuracy: ", train_accuracy[(l,k)])
        tree1

In [ ]:

tree1

In [ ]:
dataset=pd.read_csv("Indian Liver Patient Dataset (ILPD).csv",na_values='?',
            names=['age',"gender","	Total_Bilirubin ","Direct_Bilirubin","Alkphos_Alkaline_Phosphotase","Sgpt_Alamine_Aminotransferase","Sgot_Aspartate Aminotransferase","TP_Total_Protiens","ALB_Albumin","A/G_Ratio_Albumin_and_Globulin_Ratio","Selector"])
#X =np.asarray(dataset.values[:,0:dataset.shape[1]]-1,dtype=np.str)
dataset=dataset.dropna()


Y =dataset["Selector"].as_matrix()
dataset=dataset.drop("Selector",axis=1)

l=list(dataset)
l.remove("gender")
numeric_real=l
data=dataset[numeric_real].as_matrix(columns=None)
enc=KBinsDiscretizer(n_bins=3,encode='ordinal',strategy='quantile')
enc.fit(data)
X=enc.transform(data)
dataset[numeric_real]=pd.DataFrame(X,columns=numeric_real)


mutiva_att=list(dataset)

for att in mutiva_att:
    dm1=pd.get_dummies(dataset[att],drop_first=True,prefix=att)
    dataset=dataset.drop(att,axis=1)
    dataset=pd.concat([dataset,dm1],axis=1)


xt=dataset.as_matrix(columns=None)


X_train, X_test, y_train, y_test = train_test_split(xt, Y, test_size=0.33, random_state=0, stratify=Y)
C=np.unique(y_train)
lb=LabelBinarizer()
#lb.fit(C)
lb.fit(C)
y_train_ohot=lb.transform(y_train)
y_train_ohot= np.hstack((1-y_train_ohot, y_train_ohot))
y_test_ohot=lb.transform(y_test)
y_test_ohot= np.hstack((1-y_test_ohot, y_test_ohot))

N=X_train.shape[0]
M=X_train.shape[1]

In [ ]:
v=2
leaves=[6]
depths={6:[4]}
test_accuracy={}
train_accuracy={}
max_time={4:1500}
#filename='datasets/indian_liver/must_appear/indian_liver_'
for l in leaves:
    for k in depths[l]:
        tree,optimal=decisionTreeConstraint(L=l,K=k,V=v,max_time=max_time[k],X=X_train,Y=y_train_ohot,
                             colnames=list(dataset),
                        classnames=['Liver',"Not_Liver"],imbalanced=False)
        tree1=export_graphviz_cp(tree)
        y_pre_test=[predict(tree,X_test[i,:]) for i in range(X_test.shape[0])]
        y_pre_train=[predict(tree,X_train[i,:]) for i in range(X_train.shape[0])]
        test_accuracy[(l,k)]=accuracy_score(y_pre_test,np.argmax(y_test_ohot,axis=1))
        train_accuracy[(l,k)]=accuracy_score(y_pre_train,np.argmax(y_train_ohot,axis=1))
        #name=filename+'leaves='+str(l)+'_depth='+str(k)+'_maxtime='+str(max_time[k])+'_'+optimal+'_train_acc='+str(round(train_accuracy[(l,k)],5))+'_test_acc='+str(round(test_accuracy[(l,k)],5))
        tree1
        print("Leaves: ",l, ", Depth: ",k, ", test accuracy: ", test_accuracy[(l,k)],", train accuracy: ", train_accuracy[(l,k)])